## 온열환자 발생예측모형 개발을 위한 기초분석

### 온열질환자_예측모형설계_기초분석_20240618.pdf 파일

**대충 간단한 흐름 파악을 위주로 진행예정**

목적 - 온열, 한랭질환 환자의 예측을 통한 응급실 체계 관리를 위한 

기초분석 
- 매 여름 온열환자 발생 최근 5개년동안 약 11일 정도 폭염 진행
- 50대, 남성, 단순노무종사자에서 많이 발생
- 낮 시간대, 실외에서 주로 발생

EDA 
- 19~23년 데이터(8,677건)를 기반으로 분석
- 19년 대비 20~22년 환자는 감소세에서 23년에 급증(감소 시기는 코로나로 내원 줄어든 것으로 예상)
- 단순 건수로 본다면 경기도가 최다/그러나 인구대비 온열환자로 본다면 
- 19년 전남>경북>충북>제주>충남
- 23년 제주>전남>전북>경북>충남 순으로 존재
- 데이터 기간동안 여성(1920건), 남성(6757건)으로 3.5배 차이
- 실외(7007건), 실내(1670건)으로 4.2배 차이
- 직업과도 연관이 있음 단순노무 종사자의 경우가 빈도가 제일 높음
- 연령대에 따라서 보면 50대>60대>40대>30대 순
- 사망자는 환자예측과는 크게 무관해보이는데 이도 확인 필요

### 온열질환자_예측모형설계_데이터분석_20240624

데이터
- 온열환자 대상 테이블 
- ASOS 시간자료 (2019~2023)
- AWS 시간자료 (2019~2023)
- ASOS, AWS 지점 관측 위치 자료 

목적
- 환자 발생지역 인근의 기상 데이터를 매핑하여 온열환자와 기상관계를 분석
- 조건(체감온도가 높은, 기온이 높은, 습도가 높은) 

체감온도 공식을 활용하여 온열환자의 체감온도 정보 매핑

중복 매핑의 경우->해당 기간의 체감온도 파악&온도가 높은 경우를 최종관측 지점으로

TA(기온), TS(지면온도)간의 큰 차이를 보이는 경우도 존재 어떠한 것을 기준으로 할 지도 결정 필요
(지면온도가 오히려 온열환자에게 영향 미칠 수 있다는 것을 생각)




### 온열질환자_예측모형설계_데이터분석_20240702

특징변수 수정
- 체감온도(지역 그룹핑 기준 수정)
- 체감온도 가중치(시간대에 따른 가중치)
- 폭염빈도(발생일시 기준으로 3일전 매시간마다 33도를 넘은 케이스 산출(건수))
- 열대야(발생일시 기준 이전 18시 부터 해당일의 09시까지)
- x,y 격자(격자정보)

### 온열질환자_예측모형설계_모델링_20240801

환자수 예측을 위한 회귀모델(regressor)
rnn, lstm, rf, xgb//target='patientsCnt’

환자 발생 여부 예측을 위한 분류모델(classification)
rnn, lst,, rf, xgb

selectKbest 10개를 가지고 진행한 경우가 둘다 수치적으로는 더 좋음
selectKbest 
- 최적의 특성 수를 구하고 해당 특성을 이용하도록 도움
- Sikit-learn/

### 온열질환자_예측모형설계_모델링_20240808

14~21년 데이터로 학습하여 22~23년의 데이터를 예측하는데 23년의 성능이 눈에 띄게 차이남
변수의 binning을 20,15,10구간으로 구성하여
- 20구간: svm(0.81 acc), decision tree(0.75 acc), rf(0.82 acc), xgb(0.82 acc)
- 15구간: svm(0.85 acc), decision tree(0.76 acc), rf(0.85 acc), xgb(0.84 acc)
- 10구간: svm(0.84 acc), decision tree(0.78 acc), rf(0.85 acc), xgb(0.85 acc)
구간이 줄어들 수록 정확도가 올라가는 모습을 보임

군집분석(clustering)을 통해 최대기온에 따른 온열환자를 
보면 decision tree(0.78 acc), rf(0.84 acc), svm(0.83 acc), xgb(0.84 acc)

군집분석(clustering)을 통해 체감온도에 따른 온열환자를 
보면 decision tree(0.78 acc), rf(0.85 acc), svm(0.78 acc), xgb(0.84 acc)의 결과를 보임


### 온열질환자_예측모형설계_모델링_20240812

k-means 클러스트링에서 elbow 플랏을 보면 3에서 적절한 클러스터의 수를 알 수 있다

jenks natural breaks로 데이터 분포의 자연스러운 패턴에 따라 데이터를 여러 구간으로 나누면 동일하게 3개의 구간으로 나뉘는데(0건, 1~7건, 8~79건)
이를 기반으로 모델링을 하면 각 클래스 별로 영향을 주는 feature_importances가 다른 것이 보임

기본적인 모델링을 통한 정확도는 rf(0.85 acc), xgb(0.85 acc)

하이퍼파라미터 튜닝을 통해 xgb 모델에서 샘플에 가중치를 부여 feature_importances에 따라 차등하여 부여 -> 정확도는 소폭 감소했지만 1,2 클래스의 f1_score는 증가함

이후 다양한 시도중 smote로 각 클래스별 비율을 맞춰주고 oversampling을 통해 부족한 1,2 클래스의 데이터의 수를 늘림 -> 결과로 정확도도 0.89로 가장 높고 f1_score 또한 1,2 클래스가 0.5~6에서 0.8이상의 값으로 나옴 해당 실험이 가장 효과적인 튜닝으로 보임

이외 자세한 내용은 코드를 실행해보면서 각 이론에 대한 확인이 필요